# 2. ⚙️ **LangGraph Core Concepts**


LangGraph introduces a **graph-based programming model** for building LLM applications. At its core, it allows you to define **nodes**, **edges**, and **state transitions** — just like a **finite state machine**, but supercharged for LLMs.



#### 🔁 1. **Graph-based Reasoning and Control Flow**

LangGraph uses a **directed graph** where:
- **Nodes** = steps in your app (e.g., function calls, tool usage, LLM invocations)
- **Edges** = conditions or transitions between steps
- **State** = shared memory that flows through nodes and evolves over time

```python
from langgraph.graph import StateGraph

graph = StateGraph(StateType)
graph.add_node("step_1", step_1_fn)
graph.add_node("step_2", step_2_fn)
graph.add_edge("step_1", "step_2")
graph.set_entry_point("step_1")
```



#### 🧠 2. **State Management**

LangGraph revolves around **stateful applications**:
- The **state** is a Python `dict` (or `TypedDict`) that holds data passed between nodes.
- Each node can **read, update, or return** the state.

```python
def step_1(state):
    state["message"] = "Hello from step 1"
    return state
```

> This design makes your app **modular**, **testable**, and **dynamic**.



#### 🔀 3. **Conditional Branching**

LangGraph supports branching logic:
- Each node can return a **next step name** based on a condition
- Enables decision trees, routing logic, feedback loops

```python
def router(state):
    if state["intent"] == "search":
        return "search_node"
    else:
        return "fallback_node"

graph.add_conditional_edges("router", router, {
    "search_node": "search_node",
    "fallback_node": "fallback_node"
})
```



#### 🔄 4. **Looping and Recursion**

LangGraph allows **loops** natively. Nodes can route back to previous ones, creating dynamic re-evaluation and feedback-based workflows.

```python
graph.add_edge("feedback_node", "llm_agent")  # Loop back for refinement
```



#### ⚡ 5. **Streaming and Async**

LangGraph supports streaming outputs using:
```python
async for step in graph.astream(state):
    print(step)
```

Ideal for **real-time agents**, **chatbots**, or **human-in-the-loop** workflows.



#### 👥 6. **Multi-Agent Architecture (Advanced)**

You can build multi-agent systems where:
- Each agent is a node
- They communicate by passing messages through the shared state

This makes LangGraph **perfect for collaborative agent frameworks**, RAG systems, or tool-chaining.



### 🧩 Summary of Core Concepts

| Concept                 | Description                                                |
|------------------------|------------------------------------------------------------|
| **Node**               | A function that takes and returns state                    |
| **Edge**               | Defines next node(s) after the current                     |
| **State**              | A dict (or typed state) that carries context/memory        |
| **Branching**          | Conditional logic to change control flow                   |
| **Loops**              | Allow dynamic revisiting of steps/nodes                    |
| **Streaming**          | Real-time feedback from LLMs or agents                     |
| **Multi-agent**        | Architect multiple agents with message passing             |


## Graph-based reasoning and control flow

### ⚙️ **Graph-Based Reasoning and Control Flow in LangGraph (Detailed)**

LangGraph introduces a new way of designing LLM applications using **graph theory concepts**, enabling **deterministic**, **modular**, and **scalable** control over how your AI app behaves.



## 🔁 What is Graph-Based Reasoning?

Graph-based reasoning refers to designing your LLM application as a **directed graph** where:

| Graph Component | LangGraph Analogy                      |
|------------------|-----------------------------------------|
| **Node**         | A function or step (e.g., LLM prompt, tool call, API hit) |
| **Edge**         | Transition from one step to another     |
| **State**        | Shared memory that is passed and updated between nodes |
| **Entry Point**  | Start of your graph                     |
| **Conditional Edge** | Decision logic that determines next step dynamically |

This allows **visualizing and managing** the flow of logic like a **flowchart**, with **branches, loops, and terminals**.



## 🔄 Example Flow

Imagine an LLM-powered chatbot that does the following:

1. Understands intent from the user
2. Decides action (search, generate, fallback)
3. Executes the action
4. Asks for feedback
5. Optionally loops for refinement

### Graph Representation:

```
[user_input]
     ↓
[intent_classifier]
     ↓
 ┌─────────────┬─────────────┐
 │             │             │
search     generate     fallback
 │             │             │
 ↓             ↓             ↓
[feedback_check] →──(loop)──→ [user_input]
```



## 🧠 How LangGraph Implements This

### 1. **Define a State**

The state is the "memory" of the graph. You can define a TypedDict or use a Python `dict`.

```python
from typing import TypedDict

class MyState(TypedDict):
    user_input: str
    intent: str
    response: str
```

### 2. **Create Nodes**

Each node is a function that takes `state` and returns updated `state`.

```python
def classify_intent(state):
    prompt = f"What is the intent of: '{state['user_input']}'?"
    intent = some_llm(prompt)
    state["intent"] = intent
    return state
```

### 3. **Add Conditional Edges**

You can dynamically route to different nodes using **conditional edges**.

```python
def router(state):
    if state["intent"] == "search":
        return "search_node"
    elif state["intent"] == "generate":
        return "generate_node"
    else:
        return "fallback_node"
```



## 🔀 Control Flow Features

### ✅ **Sequential Flow**
Default behavior: execute nodes in order from entry to exit.

```python
graph.add_node("step1", step1_fn)
graph.add_node("step2", step2_fn)
graph.add_edge("step1", "step2")
graph.set_entry_point("step1")
```



### 🔁 **Looping**

To allow a loop (e.g., user feedback → retry), connect a later node back to a previous one.

```python
graph.add_edge("feedback_node", "user_input_node")
```



### ❓ **Branching (Conditional Edges)**

Use conditional functions to branch to different nodes.

```python
graph.add_conditional_edges("decision_node", router_function, {
    "search_node": "search_node",
    "generate_node": "generate_node",
    "fallback_node": "fallback_node"
})
```



### 🌀 **Asynchronous Streaming**

LangGraph supports real-time streaming of steps:

```python
async for output in graph.astream(state):
    print(output)
```

This is powerful for **live interactions**, such as:
- Chatbots
- Real-time agents
- Streaming tool responses



## ✅ Advantages of Graph-Based Flow

| Feature                 | Benefit                                 |
|-------------------------|------------------------------------------|
| **Visual Logic**        | Easy to understand, debug, and scale     |
| **Modular Components**  | Reusable nodes for different workflows   |
| **Dynamic Routing**     | Handle diverse user intents or data flows|
| **Feedback & Loops**    | Human-in-the-loop, retry mechanisms      |
| **Scalable Logic**      | Ideal for large agent ecosystems         |



## 🧩 Use Cases

- **Multi-agent collaboration** (each agent = a node)
- **Decision trees for business logic**
- **LLM Orchestration with retry/revision logic**
- **Human-in-the-loop review systems**
- **Chatbots with advanced context memory**



### 🔄 **Step Functions in LangGraph: Synchronous & Asynchronous**

In LangGraph, each **node** in the graph represents a **step function**. These step functions define the behavior of that part of your application, similar to how a function defines a part of a program.

LangGraph supports **both synchronous** and **asynchronous** step functions, giving you full flexibility depending on whether you're interacting with APIs, LLMs, databases, or user input.



## 🧠 What is a Step Function?

A **step function** is a Python function that:

* Accepts a `state` (usually a `dict` or TypedDict)
* Performs an operation (e.g., LLM call, tool invocation, API call)
* Returns an updated `state`

Each step is a node in the graph and is connected to others through edges.



## ✅ **Synchronous Step Functions**

These are standard Python functions that execute immediately and return output directly.

### 📦 Example:

```python
def extract_name(state):
    text = state["user_input"]
    name = text.split("My name is ")[-1]
    state["name"] = name
    return state
```

* Simple logic
* No `async` needed
* Ideal for CPU-bound or non-blocking logic



## 🔄 **Asynchronous Step Functions**

These use `async def` and allow for **non-blocking** operations. Useful when calling APIs, LLMs, or waiting on user input.

### 📦 Example:

```python
import asyncio

async def generate_response(state):
    prompt = f"Hello {state['name']}, how can I help you?"
    # Simulating an LLM/API call
    await asyncio.sleep(1)
    state["response"] = prompt
    return state
```

* Use `await` for I/O (API, LLM, database)
* Doesn’t block other operations
* Enables streaming and multi-agent scenarios



## 🧬 Mixing Synchronous & Asynchronous

LangGraph lets you mix both types:

* Sync nodes for preprocessing or simple logic
* Async nodes for LLM, web requests, or async chains

LangGraph will **automatically detect** the type and execute accordingly.


## ✅ Example Use Case

```python
from langgraph.graph import StateGraph, END

# Step functions
def get_name(state):  # sync
    name = state["user_input"].split("name is ")[-1]
    state["name"] = name
    return state

async def greet_user(state):  # async
    await asyncio.sleep(0.5)
    state["greeting"] = f"Hello {state['name']}!"
    return state

# Build graph
workflow = StateGraph(dict)
workflow.add_node("get_name", get_name)
workflow.add_node("greet_user", greet_user)
workflow.set_entry_point("get_name")
workflow.add_edge("get_name", "greet_user")
workflow.add_edge("greet_user", END)

app = workflow.compile()
```



## 🔁 Executing the Graph

### ✅ Synchronous Execution (one-shot):

```python
output = app.invoke({"user_input": "Hi, my name is Ahmad"})
print(output)
```

### 🔄 Asynchronous Streaming:

```python
async for step in app.astream({"user_input": "My name is Raza"}):
    print(step)
```

This allows **step-by-step visualization** or integration with real-time UIs like Streamlit or LangServe.



## ✅ Summary

| Feature            | Synchronous             | Asynchronous                       |
| ------------------ | ----------------------- | ---------------------------------- |
| Defined as         | `def`                   | `async def`                        |
| Use case           | Simple logic, fast ops  | API/LLM calls, I/O, delays         |
| Integration        | Just returns state      | Requires `await`, supports streams |
| Usage in LangGraph | Automatically supported | Automatically supported            |



### 🔄 **Step Functions in LangGraph: Synchronous & Asynchronous**

In LangGraph, each **node** in the graph represents a **step function**. These step functions define the behavior of that part of your application, similar to how a function defines a part of a program.

LangGraph supports **both synchronous** and **asynchronous** step functions, giving you full flexibility depending on whether you're interacting with APIs, LLMs, databases, or user input.



## 🧠 What is a Step Function?

A **step function** is a Python function that:

* Accepts a `state` (usually a `dict` or TypedDict)
* Performs an operation (e.g., LLM call, tool invocation, API call)
* Returns an updated `state`

Each step is a node in the graph and is connected to others through edges.



## ✅ **Synchronous Step Functions**

These are standard Python functions that execute immediately and return output directly.

### 📦 Example:

```python
def extract_name(state):
    text = state["user_input"]
    name = text.split("My name is ")[-1]
    state["name"] = name
    return state
```

* Simple logic
* No `async` needed
* Ideal for CPU-bound or non-blocking logic



## 🔄 **Asynchronous Step Functions**

These use `async def` and allow for **non-blocking** operations. Useful when calling APIs, LLMs, or waiting on user input.

### 📦 Example:

```python
import asyncio

async def generate_response(state):
    prompt = f"Hello {state['name']}, how can I help you?"
    # Simulating an LLM/API call
    await asyncio.sleep(1)
    state["response"] = prompt
    return state
```

* Use `await` for I/O (API, LLM, database)
* Doesn’t block other operations
* Enables streaming and multi-agent scenarios



## 🧬 Mixing Synchronous & Asynchronous

LangGraph lets you mix both types:

* Sync nodes for preprocessing or simple logic
* Async nodes for LLM, web requests, or async chains

LangGraph will **automatically detect** the type and execute accordingly.



## ✅ Example Use Case

```python
from langgraph.graph import StateGraph, END

# Step functions
def get_name(state):  # sync
    name = state["user_input"].split("name is ")[-1]
    state["name"] = name
    return state

async def greet_user(state):  # async
    await asyncio.sleep(0.5)
    state["greeting"] = f"Hello {state['name']}!"
    return state

# Build graph
workflow = StateGraph(dict)
workflow.add_node("get_name", get_name)
workflow.add_node("greet_user", greet_user)
workflow.set_entry_point("get_name")
workflow.add_edge("get_name", "greet_user")
workflow.add_edge("greet_user", END)

app = workflow.compile()
```


## 🔁 Executing the Graph

### ✅ Synchronous Execution (one-shot):

```python
output = app.invoke({"user_input": "Hi, my name is Ahmad"})
print(output)
```

### 🔄 Asynchronous Streaming:

```python
async for step in app.astream({"user_input": "My name is Raza"}):
    print(step)
```

This allows **step-by-step visualization** or integration with real-time UIs like Streamlit or LangServe.



## ✅ Summary

| Feature            | Synchronous             | Asynchronous                       |
| ------------------ | ----------------------- | ---------------------------------- |
| Defined as         | `def`                   | `async def`                        |
| Use case           | Simple logic, fast ops  | API/LLM calls, I/O, delays         |
| Integration        | Just returns state      | Requires `await`, supports streams |
| Usage in LangGraph | Automatically supported | Automatically supported            |



### 🔀 **Single-path vs. Multi-path Logic in LangGraph**

LangGraph gives you the flexibility to control **how your application flows** through different steps (or nodes) by using **graph-based logic**. This logic can be:

* **Single-path** (linear, deterministic)
* **Multi-path** (conditional, branching, dynamic)



## 1️⃣ **Single-path Logic**

In **single-path logic**, your flow always moves **from one node to the next in a fixed sequence**, like a traditional function call chain or pipeline.

### ✅ Characteristics:

* Linear flow
* No conditional branching
* Predictable and easy to debug
* Best for simple workflows (e.g., input → transform → output)

### 📦 Example:

```python
graph.add_node("step1", step1_func)
graph.add_node("step2", step2_func)
graph.set_entry_point("step1")
graph.add_edge("step1", "step2")
graph.add_edge("step2", END)
```

### 🔁 Flow:

```
step1 → step2 → END
```



## 🔀 **2️⃣ Multi-path Logic**

In **multi-path logic**, your flow can **branch into different paths** based on the result of a node. You can conditionally move to different nodes based on logic or LLM output.

### ✅ Characteristics:

* Conditional and branching
* Flexible and dynamic
* Enables tools like **decision-making agents**, **chatbots**, etc.
* Can simulate **if-else**, **switch-case**, or **FSM** (finite state machine)

### 📦 Example:

```python
def decide_next_step(state):
    if "error" in state:
        return "error_handler"
    elif state["intent"] == "get_data":
        return "fetch_data"
    else:
        return "fallback"
```

```python
graph.add_node("router", decide_next_step, is_conditional=True)
graph.add_node("fetch_data", fetch_func)
graph.add_node("error_handler", error_func)
graph.add_node("fallback", fallback_func)

graph.set_entry_point("router")
graph.add_conditional_edges("router", {
    "fetch_data": "fetch_data",
    "error_handler": "error_handler",
    "fallback": "fallback"
})
```

### 🔁 Flow:

```
router
  ├── intent: get_data ──▶ fetch_data
  ├── error in state ───▶ error_handler
  └── otherwise ───────▶ fallback
```



## 🧠 How LangGraph Executes Multi-path

LangGraph uses **"conditional nodes"** to direct flow dynamically:

* The step function must return the name of the next node.
* You define conditional edges using `add_conditional_edges()`.



## ✅ Use Case Comparison

| Feature     | Single-Path                       | Multi-Path                            |
| ----------- | --------------------------------- | ------------------------------------- |
| Flow type   | Linear                            | Conditional branching                 |
| Use case    | Fixed pipelines, ETL, summarizers | Chatbots, agents, intelligent routing |
| Flexibility | Low                               | High                                  |
| Complexity  | Simple                            | Complex but more powerful             |



## 🧪 Real-Life Example

**Goal:** If a user asks a question about sales, fetch from DB. If it's a greeting, respond. Else, fallback.

```python
def route(state):
    if "hello" in state["user_input"].lower():
        return "greet"
    elif "sales" in state["user_input"].lower():
        return "query_sales"
    else:
        return "unknown"
```

```python
graph.set_entry_point("router")
graph.add_conditional_edges("router", {
    "greet": "greet_node",
    "query_sales": "sql_node",
    "unknown": "fallback_node"
})
```





### 🧠 **State Object Design in LangGraph**

LangGraph applications use a **shared state object** to manage memory, pass variables, and maintain context as the workflow moves between nodes in the graph.

This is **core** to how LangGraph works — the state is passed from node to node and modified along the way, like a memory snapshot of your entire workflow.



## 🧱 1. What is a State Object?

A **state** in LangGraph is a Python dictionary (`dict`) that stores:

* 🧠 Memory (conversation history, intermediate results)
* 🔢 Variables (user input, LLM outputs, flags)
* ⚙️ Internal logic indicators (like routing keys, tool results)

It’s the **single source of truth** that flows through each step (node) of your graph.



## 🧬 2. How is State Passed?

Each function in a LangGraph node **receives and returns** this state.

```python
def my_node(state: dict) -> dict:
    state["step_result"] = some_function()
    return state
```

LangGraph automatically:

* Takes the returned state from one node
* Passes it to the next node



## 🛠 3. What Can the State Store?

| Type of Data      | Example Use Case                | Example Key      |
| ----------------- | ------------------------------- | ---------------- |
| User input        | Original question               | `"user_input"`   |
| LLM output        | Answer or interpretation        | `"llm_output"`   |
| Routing info      | Intent, next node to route to   | `"intent"`       |
| Intermediate data | DB results, API responses       | `"sql_result"`   |
| Chat history      | For memory + context            | `"chat_history"` |
| Error handling    | Track failed nodes, retry flags | `"error"`        |



## 🧭 4. Example: State in a Multi-Step Agent

```python
state = {
    "user_input": "Show total sales this month",
    "intent": "query_sales",
    "sql_query": "SELECT SUM(total) FROM sales_tb WHERE ...",
    "query_result": "$125,000",
    "final_answer": "Total sales this month are $125,000"
}
```

Each step appends/modifies values in this state.



## 🧮 5. Memory in State

For chatbots or agents, memory is handled by storing previous messages in state:

```python
state["chat_history"] = [
    {"role": "user", "content": "Hello"},
    {"role": "ai", "content": "Hi, how can I help you?"}
]
```

This memory can then be passed to the LLM in the prompt context.



## 🧪 6. Code Example

```python
def detect_intent(state):
    input_text = state["user_input"]
    if "price" in input_text:
        state["intent"] = "check_price"
    else:
        state["intent"] = "unknown"
    return state
```

```python
def check_price(state):
    # simulate some logic
    state["result"] = "The price is $49"
    return state
```



## 🧼 7. Tips for State Design

* 🧹 **Avoid polluting state** with too many keys — keep it clean and meaningful.
* 🧪 **Use consistent keys** (`user_input`, `intent`, `result`, `chat_history`) across your nodes.
* 🧩 **Modularize logic**: make each node update a specific part of the state.



## 🧠 Summary

| Feature                 | Description                                      |
| ----------------------- | ------------------------------------------------ |
| Shared across all nodes | Keeps memory and variables accessible everywhere |
| Dynamic updates         | Each node can read/update the state object       |
| Stateless execution     | LangGraph is stateless outside this object       |

